In [1]:

import sys
import re
import time
import string

%matplotlib inline
import numpy as np
import pandas as pd
import pre_cor
import os
from sqlalchemy import create_engine
from pandas.io import sql
import re 
import warnings

import dict_dbutils
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.externals import joblib
warnings.filterwarnings('ignore')

# 读取文件

In [2]:
#这两个文件是自行爬取的正文
file_list1 = ['20180723_revise_clearn.xlsx','20180725_clearn.xlsx']

In [3]:
cir_data_raw_1 = pd.DataFrame()
for file_name in file_list1:
    tmp_data = pd.read_excel('data/result/{0}'.format(file_name))
    print(tmp_data.shape, file_name)
    cir_data_raw_1 = pd.concat([cir_data_raw_1, tmp_data], axis = 0)

# result.to_excel('data/result/class_data_raw_1.xlsx', index = False)
print('总数：', cir_data_raw_1.shape)
cir_data_raw_1.head()

(1410, 5) 20180723_revise_clearn.xlsx
(1620, 5) 20180725_clearn.xlsx
总数： (3030, 5)


,label,title,content,url,title_url
0,噪音,做好规划不当月光族不做守财奴,一一妈妈说理财·出品\n (图片)\n\r月光族困惑：···这个月才过半，各种经费不够啊！...,https://xueqiu.com/1983578372/98551727,妈妈说理财: 做好规划不当月光族不做守财奴 一一妈妈说理财·出品 月光族困惑：···这个月才...
1,噪音,做车险代理的前景如何 谊信合让你的财富事业迈上新高度,小本创业网\n\r»\n\r加盟项目资讯›\n\r»\n\r资讯中心\n\r»\n\r做车...,http://jxhb.91cy.cn/119134.shtml,做车险代理的前景如何 谊信合让你的财富事业迈上新高度
2,产品销售,坐飞机必须买保险？航意险适合哪些人？,坐飞机必须买保险？航意险适合哪些人？\n\r 随着经济的发展，大家对生活品质的追求，越来...,http://www.dtnews.cn/2018/january/99502C93.html,坐飞机必须买保险？航意险适合哪些人？\r\n - 大同新闻网
3,噪音,左权县人民政府金融工作办公室2017年统筹整合财政资金项目公示,\n\r为推进我县\n“五位一体”扶贫小额信贷工作，各承贷银行对全县建档立卡贫困户累计备案...,http://www.zqxzf.com/newscont.asp?id=61811&pid...,左权县人民政府金融工作办公室2017年统筹整合财政资金项目公示
4,消费服务,昨天车险报案比平常翻倍,(图片)\n\r雪中骑行滑倒 朱小军 摄\n\r 一场暴雪让扬州各家保险公司忙翻了天，车...,http://3g.k.sohu.com/t/n253854349,昨天车险报案比平常翻倍


In [4]:
#这些文件是自身带有正文的文件  20180727_circ_class_1.xls中合并了7.30号监管类补录数据
file_list2 = ['20180727_circ_class_1.xlsx']

In [5]:
cir_data_raw_2 = pd.DataFrame()
for file_name in file_list2:
    tmp_data = pd.read_excel('data/raw_data/{0}'.format(file_name))
    print(tmp_data.shape, file_name)
    cir_data_raw_2 = pd.concat([cir_data_raw_2, tmp_data], axis = 0)
    
print('总数：', cir_data_raw_2.shape)
cir_data_raw_2.head()

(4478, 4) 20180727_circ_class_1.xlsx
总数： (4478, 4)


,number,title,content,label
0,1,做人干干净净，善良终有好报！,牵手国寿 中国人寿，一言九鼎；相知多年，值得托付！ 做人，就两个字：善良。 善良的人，永远都...,产品销售
1,2,做个好人心正身安魂梦稳 行些善事天知地鉴鬼神钦 以人为本 以天为则,完整财富=家庭 健康 教育 事业 服务 金钱 精神 传播聚财 守财 传财 散财 善财之道 家...,产品销售
2,3,昨天这对夫妻的聊天记录曝光了，网友们都炸了，女人，要不娘家有钱，要不自己有保险...,爱华云服务 让生活更美好，让华夏充满爱！ 关注 什么是女人的安全感？不是什么漂亮的...,产品销售
3,4,尊享代驾 安全快捷送您回家,为回馈广大客户，富德生命人寿第十二届客服节期间特意为客户提供了免费补发保单、尊享代驾等增值服...,其他相关报道
4,5,尊享e生旗舰版震撼升级！上海质子重离子医院治疗也可100%报销！！！,各位亲爱的会员，国民医保再次升级，现在购买尊享e生旗舰版，可自主增加质子重离子医院治...,产品销售


In [6]:
cir_data_raw = pd.concat([cir_data_raw_1[['title', 'content', 'label']], 
                          cir_data_raw_2[['title', 'content', 'label']]], 
                         axis = 0)
print(cir_data_raw.shape)
cir_data_raw.head()

(7508, 3)


,title,content,label
0,做好规划不当月光族不做守财奴,一一妈妈说理财·出品\n (图片)\n\r月光族困惑：···这个月才过半，各种经费不够啊！...,噪音
1,做车险代理的前景如何 谊信合让你的财富事业迈上新高度,小本创业网\n\r»\n\r加盟项目资讯›\n\r»\n\r资讯中心\n\r»\n\r做车...,噪音
2,坐飞机必须买保险？航意险适合哪些人？,坐飞机必须买保险？航意险适合哪些人？\n\r 随着经济的发展，大家对生活品质的追求，越来...,产品销售
3,左权县人民政府金融工作办公室2017年统筹整合财政资金项目公示,\n\r为推进我县\n“五位一体”扶贫小额信贷工作，各承贷银行对全县建档立卡贫困户累计备案...,噪音
4,昨天车险报案比平常翻倍,(图片)\n\r雪中骑行滑倒 朱小军 摄\n\r 一场暴雪让扬州各家保险公司忙翻了天，车...,消费服务


In [7]:
cir_data_raw.groupby('label')['title'].count()#‘噪音 ’ ‘噪音’会引起标签名不同  直接分组统计存在问题

label
产品销售       320
公司内部管理     584
其他相关报道     678
噪音        3370
噪音           6
消费服务       281
监管        1488
行业         317
资本市场       463
Name: title, dtype: int64

# 对标题 正文 和标签进行处理

In [7]:
contents=[re.sub(r'n+ r*', '', x) for x in pre_cor.handle_contents(cir_data_raw_1['content'].tolist())]+pre_cor.handle_contents(cir_data_raw_2['content'].tolist())#去除爬取的正文所包含\n、\r
titles= pre_cor.handle_contents(cir_data_raw_1['title'].tolist()+cir_data_raw_2['title'].tolist())
labels=cir_data_raw_1['label'].tolist()+cir_data_raw_2['label'].tolist()
print(len(contents),len(titles),len(labels))


7508 7508 7508


In [8]:
#合并标记与正文
title_contents=[]
for i in range(len(titles)):
    title_contents.append(titles[i]+'title_contents '+contents[i])
print(len(title_contents))

7508


# 分别保存正文、标题、标题+正文的分词结果 以及对应 标签

In [9]:
for i,name in enumerate([ '监管','行业','产品销售','资本市场','公司内部管理','消费服务','其他相关报道','噪音']):
    print('{0}:   {1}'.format(name,labels.count(name)/len(labels)))

监管:   0.1981885988279169
行业:   0.042221630261054875
产品销售:   0.042621204049014386
资本市场:   0.06166755460841769
公司内部管理:   0.07778369738945125
消费服务:   0.03742674480554076
其他相关报道:   0.09030367607884923
噪音:   0.44898774640383593


In [10]:
coprus_save_filename = 'data/result/all_contents_pre.txt'
f = open(coprus_save_filename, "w+", encoding='UTF-8')
for data in contents:
    f.write(data + '\n')
f.close()

In [11]:
titles_save_filename= 'data/result/all_titles_pre.txt'
ff = open(titles_save_filename, "w+", encoding='UTF-8')
for data in titles:
    ff.write(data + '\n')
ff.close()

In [12]:
title_contents_save_filename= 'data/result/all_titles+contents_pre.txt'
ff = open(title_contents_save_filename, "w+", encoding='UTF-8')
for data in title_contents:
    ff.write(data + '\n')
ff.close()

In [13]:
labels_save_filename= 'data/result/all_labels_pre.txt'
fff = open(labels_save_filename, "w+", encoding='UTF-8')
for data in labels:
    fff.write(data + '\n')
fff.close()

# 每个类别的关键词提取

样例：
from jieba import analyse
#引入TextRank关键词抽取接口
textrank = analyse.textrank

#原始文本
text = "线程是程序执行时的最小单位，它是进程的一个执行流，\
        是CPU调度和分派的基本单位，一个进程可以由很多个线程组成，\
        线程间共享进程的所有资源，每个线程有自己的堆栈和局部变量。\
        线程由CPU独立调度执行，在多CPU环境下就允许多个线程同时运行。\
        同样多线程也可以实现并发操作，每个请求分配一个线程来处理。"

print("\nkeywords by textrank:")
#基于TextRank算法进行关键词抽取
keywords = textrank(text,topK=5)
#输出抽取出的关键词
for keyword in keywords:
    print(keyword + "/",)

In [11]:
titles_1= pre_cor.handle_contents(cir_data_raw_1[cir_data_raw_1["label"]=='监管']['title'].tolist()+cir_data_raw_2[cir_data_raw_2["label"]=='监管']['title'].tolist())
contents_1=[re.sub(r'n+ r*', '', x) for x in pre_cor.handle_contents(cir_data_raw_1[cir_data_raw_1["label"]=='监管']['content'].tolist())]+pre_cor.handle_contents(cir_data_raw_2[cir_data_raw_2["label"]=='监管']['content'].tolist())#去除爬取的正文所包含\n、\r
print(len(titles_1),len(contents_1))

1488 1488


In [8]:
titles_1= cir_data_raw_1[cir_data_raw_1["label"]=='监管']['title'].tolist()+cir_data_raw_2[cir_data_raw_2["label"]=='监管']['title'].tolist()
contents_1=[re.sub(r'n+ r*', '', x) for x in cir_data_raw_1[cir_data_raw_1["label"]=='监管']['content'].tolist()]+cir_data_raw_2[cir_data_raw_2["label"]=='监管']['content'].tolist()#去除爬取的正文所包含\n、\r
print(len(titles_1),len(contents_1))

1488 1488


In [9]:
from jieba import analyse
analyse.set_stop_words("corpus/stop_words_cor.txt")
textrank = analyse.textrank
only_word_1=set()
only_word_dict_1={}
for i in range(len(titles_1)):
    keywords = textrank(titles_1[i]+' '+contents_1[i],topK=5)
    only_word_1=only_word_1|set(keywords)
    for word in keywords:
        if word in only_word_dict_1:
            only_word_dict_1[word]=only_word_dict_1[word]+1
        else:
            only_word_dict_1[word]=1 
print(only_word_1)
item_list_1=sorted(only_word_dict_1.items(),key = lambda x:x[1],reverse = True)
print(item_list_1)

{'娱乐', '出表', '余额', '参保', '证券', '分公司', '排查', '工作人员', '巨灾', '整改', '英国', '减保', '信托', '能力', '指标', '征求意见', '深圳', '变相', '家庭', '链接', '金融危机', '加强', '破产', '给付', '递延', '后续', '驾车', '保单', '稳赢', '不法', '表外', '费率', '辅助', '民生', '民营企业', '金融服务', '借贷', '建设工程', '继续', '聘任', '机器', '回表', '工作', '金融', '责令', '前进', '新增', '指导', '新疆', '监管部门', '系数', '设置', '银行业', '会员', '例行', '推荐', '游戏', '就业', '处罚', '流动性', '绿色', '专门', '金融街', '平台', '苏州', '个人资料', '估损', '时任', '财经', '关联方', '子站', '资料', '科学化', '网站', '推动', '治乱', '比例', '要求', '行情', '重点', '计划', '品种', '人气', '粤港澳', '当事人', '交叉', '社会主义', '汽车', '中间业务', '计量', '鄞州区', '公布', '考核', '商报', '延误', '安责险', '货物', '方面', '人身险', '订阅', '账户', '联合', '变现', '债券市场', '内需', '退休', '问询', '支持', '法院', '相关', '投资人', '案件', '认为', '退旧', '收购', '下修', '变化', '落地', '航班', '发现', '公司', '负债', '事务所', '数字', '事件', '问题', '设计', '现金', '黄金', '解读', '罚单', '常州', '反洗钱', '国华人寿', '经纬', '开放', '投保人', '库存', '综合', '干部', '抽查', '规模', '扬州', '意义', '披露', '登记', '总书记', '占比', '支公司', '发布', '召开', '打击', '确认', '资产', '中介机构', '村民', '代理', '边际', '约定', '存

In [36]:
titles_2= pre_cor.handle_contents(cir_data_raw_1[cir_data_raw_1["label"]=='行业']['title'].tolist()+cir_data_raw_2[cir_data_raw_2["label"]=='行业']['title'].tolist())
contents_2=[re.sub(r'n+ r*', '', x) for x in pre_cor.handle_contents(cir_data_raw_1[cir_data_raw_1["label"]=='行业']['content'].tolist())]+pre_cor.handle_contents(cir_data_raw_2[cir_data_raw_2["label"]=='行业']['content'].tolist())#去除爬取的正文所包含\n、\r
print(len(titles_2),len(contents_2))

317 317


In [10]:
titles_2= cir_data_raw_1[cir_data_raw_1["label"]=='行业']['title'].tolist()+cir_data_raw_2[cir_data_raw_2["label"]=='行业']['title'].tolist()
contents_2=[re.sub(r'n+ r*', '', x) for x in cir_data_raw_1[cir_data_raw_1["label"]=='行业']['content'].tolist()]+cir_data_raw_2[cir_data_raw_2["label"]=='行业']['content'].tolist()#去除爬取的正文所包含\n、\r
print(len(titles_2),len(contents_2))

317 317


In [11]:
only_word_2=set()
only_word_dict_2={}
for i in range(len(titles_2)):
    keywords = textrank(titles_2[i]+' '+contents_2[i],topK=5)
    only_word_2=only_word_2|set(keywords)
    for word in keywords:
        if word in only_word_dict_2:
            only_word_dict_2[word]=only_word_dict_2[word]+1
        else:
            only_word_dict_2[word]=1 
print(only_word_2)
item_list_2=sorted(only_word_dict_2.items(),key = lambda x:x[1],reverse = True)
print(item_list_2)

{'下滑', '重庆', '参保', '出游', '委员会', '证券', '分公司', '儿童', '排查', '蓬溪县', '高端', '上榜', '信托', '展业', '圆桌', '指标', '家庭', '遗产税', '加强', '给付', '昆明', '递延', '保单', '开封', '费率', '民生', '家长', '面临', '时评', '继续', '补助', '医生', '核心', '工作', '金融', '利润', '银行业', '会计师', '处罚', '房东', '江西', '报案', '绿色', '导致', '平台', '预警', '壽險', '演练', '雅安市', '陕西省', '诉讼', '浙江省', '示范区', '计划', '粤港澳', '汽车', '手续费', '方面', '人身险', '环境污染', '保险业务', '订阅', '水产', '法院', '支持', '案件', '地震', '长春市', '公司', '拥有', '场景', '问题', '烟台市', '投保人', '专属', '综合', '规模', '险种', '交流', '发布', '报价', '资产', '代理', '存在', '数据', '有所', '实践', '主题', '天然橡胶', '实名', '保险经纪', '销售', '旗下', '众安', '灾害', '美国', '时间', '车险', '致贫', '全省', '调整', '债券', '大河', '养老保险', '公告', '管理', '评级', '名义', '保單', '财经网', '累计', '应该', '保持', '举报', '保险业', '申请', '选择', '政策性', '集团', '祥云', '外贸', '云南', '产能', '完善', '癌症', '纠纷', '分期', '出险', '人壽', '品牌', '持股', '股份', '行长', '试点', '债权', '收益率', '资本', '农业', '全市', '游客', '本站', '月份', '总裁', '赔款', '风险', '提升', '专业', '唐山市', '资产负债', '山东', '履约', '小幅', '收益', '开始', '农民', '住院', '经营', '单月', '太仓', '准备金', '牌照',

In [38]:
titles_3= pre_cor.handle_contents(cir_data_raw_1[cir_data_raw_1["label"]=='消费服务']['title'].tolist()+cir_data_raw_2[cir_data_raw_2["label"]=='消费服务']['title'].tolist())
contents_3=[re.sub(r'n+ r*', '', x) for x in pre_cor.handle_contents(cir_data_raw_1[cir_data_raw_1["label"]=='消费服务']['content'].tolist())]+pre_cor.handle_contents(cir_data_raw_2[cir_data_raw_2["label"]=='消费服务']['content'].tolist())#去除爬取的正文所包含\n、\r
print(len(titles_3),len(contents_3))

281 281


In [13]:
titles_3= cir_data_raw_1[cir_data_raw_1["label"]=='消费服务']['title'].tolist()+cir_data_raw_2[cir_data_raw_2["label"]=='消费服务']['title'].tolist()
contents_3=[re.sub(r'n+ r*', '', x) for x in cir_data_raw_1[cir_data_raw_1["label"]=='消费服务']['content'].tolist()]+cir_data_raw_2[cir_data_raw_2["label"]=='消费服务']['content'].tolist()#去除爬取的正文所包含\n、\r
print(len(titles_3),len(contents_3))

281 281


In [14]:
only_word_3=set()
only_word_dict_3={}
for i in range(len(titles_3)):
    keywords = textrank(titles_3[i]+' '+contents_3[i],topK=5)
    only_word_3=only_word_3|set(keywords)
    for word in keywords:
        if word in only_word_dict_3:
            only_word_dict_3[word]=only_word_dict_3[word]+1
        else:
            only_word_dict_3[word]=1 
print(only_word_3)
item_list_3=sorted(only_word_dict_3.items(),key = lambda x:x[1],reverse = True)
print(item_list_3)

{'页面', '重庆', '病有所医', '马铃薯', '关系', '朋友', '自驾', '内丘', '地道', '查验', '用药', '分公司', '火灾', '工作人员', '驾驶', '律师', '受灾', '社区', '理财', '大豆', '鉴定', '驴友', '增值', '家庭', '台湾', '肇事', '大棚', '检查', '登录', '给付', '养殖户', '获赞', '气候', '高质量', '合作', '保单', '开封', '跨境', '次数', '民生', '涉水', '规定', '成立', '三门峡', '治疗', '华夏', '意外事故', '双方', '武器', '台风', '继续', '济困', '办理', '赔付', '瓯海', '行驶', '工作', '金融', '有限公司', '定损', '政府', '平安保险', '小组', '阿克苏', '消费者', '航行', '安徽', '支农', '湖州', '推荐', '业绩', '贫困户', '4S店', '报案', '江西', '显示', '保额', '处罚', '导致', '车主', '治安', '平台', '堤防', '保险合同', '苏州', '总部', '保员', '骗保', '葫芦岛', '交通事故', '财经', '社会', '事情', '科技', '查勘', '网友', '资料', '依法', '诉讼', '女士', '水稻', '寿县', '支付', '要求', '太平', '费用', '受损', '芦笋', '当事人', '汽车', '安比', '发生', '建国', '金融保险', '恩施', '内江市', '商报', '忻州', '工业园区', '淮南', '投保', '外国', '中原', '北分', '客户', '贷款', '存钱', '砍断', '保险', '没有', '轻症', '赔偿', '配件', '水产', '温州', '承诺', '法院', '种植', '投诉', '中山', '体检', '患者', '相关', '案件', '涉及', '赛事', '柜面', '万宁市', '身故', '查询', '发现', '公司', '先生', '进行', '事件', '车子', '六盘水', '中华人民共和国', '侧壁', '材料', '编

In [15]:
titles_4= cir_data_raw_1[cir_data_raw_1["label"]=='产品销售']['title'].tolist()+cir_data_raw_2[cir_data_raw_2["label"]=='产品销售']['title'].tolist()
contents_4=[re.sub(r'n+ r*', '', x) for x in cir_data_raw_1[cir_data_raw_1["label"]=='产品销售']['content'].tolist()]+cir_data_raw_2[cir_data_raw_2["label"]=='产品销售']['content'].tolist()#去除爬取的正文所包含\n、\r
print(len(titles_4),len(contents_4))
only_word_4=set()
only_word_dict_4={}
for i in range(len(titles_4)):
    keywords = textrank(titles_4[i]+' '+contents_4[i],topK=5)
    only_word_4=only_word_4|set(keywords)
    for word in keywords:
        if word in only_word_dict_4:
            only_word_dict_4[word]=only_word_dict_4[word]+1
        else:
            only_word_dict_4[word]=1 
print(only_word_4)
item_list_4=sorted(only_word_dict_4.items(),key = lambda x:x[1],reverse = True)
print(item_list_4)

320 320
{'女性', '页面', '拉伤', '疾险', '分红', '派发', '关系', '儿童', '评选', '高端', '飞机', '驾驶', '社区', '理财', '保护', '信托', '标准', '汽车保险', '停售', '学平险', '家庭', '宁波市', '病房', '破产', '给付', '后续', '受益', '保单', '合作', '不可', '次数', '费率', '了免', '姓名', '溢金', '规定', '陌生人', '华夏', '治疗', '类产品', '行驶证', '肌肉', '适合', '证件号码', '线下', '办理', '赔付', '部分', '工作', '输入', '金融', '有限公司', '平安保险', '包含', '系数', '消费者', '等待', '年金', '会员', '豁免', '推荐', '业绩', '税务', '保额', '车主', '醒目', '经代', '平台', '迎来', '团购', '壽險', '奋战', '行天下', '大童', '保通', '提前', '计生', '尊传', '交通事故', '生病', '社会', '科技', '本合同', '互助', '女士', '网站', '支付', '推动', '服务平台', '增利', '比例', '要求', '计划', '费用', '商业', '保证', '太平', '摔倒', '系统性', '营口', '补偿', '互联网', '汽车', '大家', '重疾', '发生', '评论', '是否', '传承', '投保', '堆集', '保险业务', '分类', '账户', '客户', '贷款', '联合', '组合', '保险', '轻症', '没有', '赔偿', '探索', '单证', '苹果', '支持', '差评', '世纪', '老友', '年龄', '相关', '补贴', '借款人', '身故', '查询', '公司', '事务所', '数字', '进行', '意向', '问题', '现金', '材料', '业务', '原发性', '女人', '开发票', '发票', '租赁', '总额', '疾病', '生存', '人人', '维修', '金额', '乘坐', '投保人', '指南', '综合', '专属', '附

from  textrank4zh import TextRank4Keyword,TextRank4Sentence
TextRank = TextRank4Keyword(stop_words_file='corpus/stopwords.txt')
for talk in corpus_AAB_txt:
    wordlist=''
    TextRank.analyze(talk,window = 2,lower = True,vertex_source='no_stop_words',edge_source='no_stop_words')
    keywords = TextRank.get_keywords(num = 2,word_min_len = 1)#获取最重要的num个长度大于等于word_min_len的关键词。
    #talk_cut=TextRank.get_wordlist(vertex_source='no_stop_words')
    #wordslist.append(talk_cut)
    print('关键词:')
    for w in w_list:
        print(w.word,w.weight) 

# 导出模型分类错输的数据

In [47]:
class StatsFeatures_cor(BaseEstimator, TransformerMixin):
    
    def __init__(self):
        self.neg = set()
        f = open("corpus/neg_words.txt","r+", encoding='UTF-8')
        for content in f:
            self.neg.add(content)
        f.close()
        dict_org=dict_dbutils.get_dicts()
        self.org=set(dict_org.keys())
        
        

    def fit(self, X, y=None):
        return self

    def getcnt(self,x):        
        return len(list(set(x)))

    def getnegcnt(self,x):
        negcnt = 0
        words = x.split()
        for w in words:
            if w in self.neg:
                negcnt = negcnt+1
        return negcnt
    def getorgcnttf(self,x):
        orgcnt=0
        orgtf=0
        words = x.split()
        words_set=set(words)
        for w in words_set:
            if w in self.org:
                orgcnt = orgcnt+1
                orgtf=orgtf+words.count(w)
        if(orgcnt>0):
            return orgcnt,orgtf
        else:
            return orgcnt,orgtf
    
    
    def transform(self, X):
        data = []
        for x in X:
            if len(x) == 0:
                length  = 1
            else :
                length = len(x)
            orgcnt,orgtf=self.getorgcnttf(x)
            data.append([len(x),self.getcnt(x),self.getcnt(x)/length,
                         self.getnegcnt(x),self.getnegcnt(x)/length,
                         orgcnt,orgtf])            
        return data
pipeline=joblib.load("model/circ_8classifier_0731.pkl.z")

In [48]:
label_dic={'监管':1,'行业':2,'产品销售':3,'资本市场':4,'公司内部管理':5,'消费服务':6,'其他相关报道':7,'噪音':8}
label_dic_turn={1:'监管',2:'行业',3:'产品销售',4:'资本市场',5:'公司内部管理',6:'消费服务',7:'其他相关报道',8:'噪音'}

In [49]:
label=[]
label=[label_dic[x.strip()] for x in labels]
print(len(title_contents),len(label))

7508 7508


In [50]:
from sklearn import metrics
y_pred_class = pipeline.predict(title_contents)
print('accuracy_score: ', metrics.accuracy_score(label, y_pred_class)) # 指所有分类正确的百分比
print(metrics.classification_report(label, y_pred_class))
print('confusion_matrix: ')
print( metrics.confusion_matrix(label, y_pred_class))

accuracy_score:  0.9802876931273309
             precision    recall  f1-score   support

          1       0.98      0.99      0.98      1488
          2       0.96      0.91      0.94       317
          3       0.98      0.93      0.95       321
          4       0.99      0.97      0.98       461
          5       0.97      0.97      0.97       580
          6       0.99      0.98      0.98       280
          7       0.98      0.98      0.98       679
          8       0.98      0.99      0.99      3382

avg / total       0.98      0.98      0.98      7508

confusion_matrix: 
[[1469    3    1    0    4    0    1   10]
 [   9  290    0    2    2    1    2   11]
 [   1    3  299    0    2    0    2   14]
 [   3    1    0  445    1    0    0   11]
 [   6    1    3    0  561    0    4    5]
 [   1    0    0    0    2  274    0    3]
 [   0    2    1    0    2    2  666    6]
 [  11    3    2    4    3    0    3 3356]]


In [51]:
df_empty = pd.DataFrame(columns=['title', 'content', 'label', 'predict_labels'])
df_empty['title']=cir_data_raw_1['title'].tolist()+cir_data_raw_2['title'].tolist()
df_empty['content']=cir_data_raw_1['content'].tolist()+cir_data_raw_2['content'].tolist()
df_empty['label']=[x.strip() for x in labels]
df_empty['predict_labels']=[label_dic_turn[x] for x in y_pred_class]
df_empty.head()

,title,content,label,predict_labels
0,做好规划不当月光族不做守财奴,一一妈妈说理财·出品\n (图片)\n\r月光族困惑：···这个月才过半，各种经费不够啊！...,噪音,噪音
1,做车险代理的前景如何 谊信合让你的财富事业迈上新高度,小本创业网\n\r»\n\r加盟项目资讯›\n\r»\n\r资讯中心\n\r»\n\r做车...,噪音,噪音
2,坐飞机必须买保险？航意险适合哪些人？,坐飞机必须买保险？航意险适合哪些人？\n\r 随着经济的发展，大家对生活品质的追求，越来...,产品销售,产品销售
3,左权县人民政府金融工作办公室2017年统筹整合财政资金项目公示,\n\r为推进我县\n“五位一体”扶贫小额信贷工作，各承贷银行对全县建档立卡贫困户累计备案...,噪音,噪音
4,昨天车险报案比平常翻倍,(图片)\n\r雪中骑行滑倒 朱小军 摄\n\r 一场暴雪让扬州各家保险公司忙翻了天，车...,消费服务,消费服务


In [52]:
result=df_empty[df_empty['label']!=df_empty['predict_labels']]
result.to_excel('data/result/predict_wrong.xlsx', index = False)
result.head()

,title,content,label,predict_labels
48,重庆11部门联合出台金融精准扶贫政策,原标题：\n\r11部门联手推进金融精准扶贫\n\r助力重庆深度贫困地区脱贫攻坚\n\r本...,监管,噪音
65,众安在线与平安财险平分车险业务保费,车险业务将覆盖\n\r除港澳台之外所有区域\n\r虽然2015年就推出保骉车险，但因为遇到...,公司内部管理,产品销售
132,——中国人寿保险股份有限公司2017年保费突破5000亿大关,（原标题：——中国人寿保险股份有限公司2017年保费突破5000亿大关）\n\r(图片)\...,公司内部管理,其他相关报道
179,政策利好健康险 保费持续稳增,近年来，\n健康险\n保持快速增长，保费收入增速远高于寿险保费收入增速。截至2016年底，...,行业,监管
372,铁路乘客意外伤害保障计划保险条款有哪些？,最佳答案：\n\r1、短期交通人身意外伤害保险（2013版）条款-太平洋产险\n\r2、附...,噪音,产品销售


In [53]:
print(result.shape)

(148, 4)
